In [1]:
from sklearn import tree, pipeline, preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier


from sklearn import datasets, linear_model, metrics 

from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

Пример как должен выглядеть ответ

In [5]:
example_of_answer = pd.read_csv('/Users/avyny/Desktop/abc/submission_example.csv')
example_of_answer.head()

,user_id,is_gone
0,12,0.26
1,13,0.79
2,15,0.12
3,21,0.67
4,35,0.62


Мы будем считать, что пользователь успешно закончил курс, если он правильно решил больше 40 практических заданий.

В данных:

- submission_data_test.csv
- events_data_test.csv

хранится информация о решениях и действиях для 6184 студентов за первые два дня прохождения курса. Это 6184 студентов, которые проходили курс в период с мая 2018 по январь 2019. Подробное описание переменных смотри в этом шаге.  

Используя данные о первых двух днях активности на курсе вам нужно предсказать, наберет ли пользователь более 40 баллов на курсе или нет.

В этих данных, вам доступны только первые два дня активности студентов для того, чтобы сделать предсказание. На самом деле, используя эти данные, вы уже можете сделать прогноз. Например, если пользователь за первые два дня набрал 40 баллов, скорее всего он наберет более 40 баллов в дальнейшем. Чтобы подкрепить такие гипотезы, вы можете использовать данные, на которые мы исследовали в первых двух модулях курса, где для всех пользователей представлены все данные об их активности на курсе. 

In [25]:
submissions_data = pd.read_csv('/Users/avyny/Desktop/abc/submission_data_test.csv')

var1 = ((submissions_data.groupby('user_id').timestamp.max() - submissions_data.groupby('user_id').timestamp.min())/(60 * 60))
var1 = pd.DataFrame(var1).reset_index()
# var1.query("timestamp > 1.5")

users_scores = submissions_data.pivot_table(index = "user_id", columns = "submission_status", values = "step_id", aggfunc="count", fill_value=0) \
           .reset_index()
users_scores = users_scores.merge(var1, how = "outer")
# users_scores['more_than_dd'] = (users_scores.timestamp >= 1.6)
# users_scores = users_scores.drop('timestamp', axis=1)
# users_scores.query('correct > 10 & wrong == 0')
users_scores.query('correct < 40 & correct >= 8 & wrong <= 1 & timestamp < 0.7')   #им будет легко - врядли они сдадут

,user_id,correct,wrong,timestamp
295,2866,28,0,0.193889
690,6650,9,1,0.277222
1336,12435,9,1,0.372778
1536,14412,9,1,0.584167
1734,16354,9,1,0.666944
2108,20137,9,0,0.049167
2460,23611,27,0,0.250278
2615,25030,26,0,0.253611


In [27]:
events_data = pd.read_csv('/Users/avyny/Desktop/abc/events_data_test.csv')
var2 = ((events_data.groupby('user_id').timestamp.max() - events_data.groupby('user_id').timestamp.min())/(60 * 60 * 24))
users_events_data = events_data.pivot_table(index = "user_id", columns = "action", values = "step_id", aggfunc="count", fill_value=0) \
           .reset_index()
users_events_data.head()

action,user_id,discovered,passed,started_attempt,viewed
0,4,1,1,0,1
1,6,1,1,0,1
2,10,2,2,0,6
3,12,11,9,4,14
4,13,70,70,35,105


In [5]:
# var_time = (events_data.groupby('user_id')['timestamp'].max() - events_data.groupby('user_id')['timestamp'].min())/(24*60*60)
# var_time = pd.DataFrame({'total_time': var_time}).reset_index()
# var_time.head()
# var_time.query("total_time > 1.5").count()

In [845]:
final_table = users_scores.merge(users_events_data, how = 'outer')
final_table = final_table.fillna(0)
final_table.query('user_id == 1619')

,user_id,correct,wrong,timestamp,discovered,passed,started_attempt,viewed
171,1619,3.0,0.0,0.024722,1,1,0,1


In [892]:
# final_table['ratio'] = final_table.correct/(final_table.wrong + 0.1)
final_table['is_stay_user'] = ((final_table.correct > 10.0) & (final_table.wrong >= 1) & (final_table.timestamp > 0.8) ) | (final_table.correct >= 40) 
#     | ((final_table.correct == 3) & (final_table.correct != 0) & (final_table.timestamp != 0) & (final_table.timestamp < 0.10) )


X = final_table.drop('is_stay_user', axis=1)
X = X.set_index(X.user_id)
X = X.drop(['user_id'], axis=1)
y = final_table.is_stay_user.map(int)

y.sum()


601

In [893]:
# np.percentile(X.correct, 91)

In [894]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True)

In [895]:

# numeric_data_columns = ['correct', 'wrong', 'discovered', 'passed', 'started_attempt', 'viewed']
# numeric_data_indices = np.array([(column in numeric_data_columns) for column in X_train.columns], dtype = bool)


In [896]:
# regressor = RandomForestClassifier()
# estimator = pipeline.Pipeline(steps = [       
#     ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
#                         #numeric
#             ('numeric_variables_processing', pipeline.Pipeline(steps = [
#                 ('selecting', preprocessing.FunctionTransformer(lambda data: data)),
#                 ('scaling', preprocessing.StandardScaler(with_mean = 0, with_std = 1))            
#                         ]))
#         ])),
#     ('model_fitting', regressor)
#     ]
# )

In [897]:
# estimator.fit(X, y)

In [898]:
best_tree = RandomForestClassifier(n_estimators = 10, max_depth = 8, criterion = 'entropy', min_samples_split = 5, min_samples_leaf = 5)
best_tree.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [899]:
test_predictions = best_tree.predict(X)
print(test_predictions)

[0 1 0 ... 0 0 0]


In [900]:
best_tree.predict_proba(X)

array([[1. , 0. ],
       [0. , 1. ],
       [0.9, 0.1],
       ...,
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ]])

In [901]:
best_tree.feature_importances_

array([0.46778544, 0.04825771, 0.0519918 , 0.06205996, 0.30433611,
       0.06183175, 0.00373723])

In [149]:
parameters = {'n_estimators': range(10, 50, 10),
             'max_depth': range(3, 10),
             'criterion': ['gini', 'entropy'],
             'min_samples_split': range(5, 20, 5),
             'min_samples_leaf': range(5, 20, 5)
             }
grid_search = GridSearchCV(regressor, param_grid = parameters, cv = 5)
grid_search.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [150]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9964424320827943
{'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 10}


In [151]:
test_predictions = grid_search.best_estimator_.predict(X)
print(test_predictions)

[0 1 0 ... 0 0 0]


In [152]:
best_tree = grid_search.best_estimator_
best_tree.predict_proba(X)

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.83320982, 0.16679018],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [153]:
best_tree = grid_search.best_estimator_
probability_predictions = best_tree.predict_proba(X)
probability_predictions

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.83320982, 0.16679018],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [154]:
best_tree.feature_importances_

array([0.39301405, 0.03868021, 0.10402319, 0.15951825, 0.16805774,
       0.10098855, 0.03571801])

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True)

classifier = linear_model.SGDClassifier(loss = 'log', random_state = 1)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
probability_predictions = classifier.predict_proba(X)

matrix = metrics.confusion_matrix(y_test, predictions)

# print(metrics.classification_report(y_test, predictions))

# metrics.roc_auc_score(y_test, predictions)

probability_predictions[:,1]
# classifier.score(X, y)

array([0.00000000e+000, 1.00000000e+000, 2.42452234e-287, ...,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000])

In [241]:
# best_tree = RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_leaf=10, min_samples_split=15, n_estimators=20)
clf = RandomForestClassifier()
parameters = {'n_estimators': range(10, 50, 10),
             'criterion': ['gini', 'entropy'],
             'max_depth': range(2, 6),
             'min_samples_split': range(5, 20, 5),
             'min_samples_leaf': range(5, 20, 5)
             }
grid_search = GridSearchCV(clf, param_grid = parameters, n_jobs = -1, cv = 5)
grid_search.fit(X, y)
best_tree = grid_search.best_estimator_
best_tree

# best_tree.fit(X, y)

# RandomForestClassifier: criterion='entropy', max_depth=5, min_samples_leaf=10, min_samples_split=10, n_estimators=20

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=15, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [242]:
# predictions2 = best_tree.predict(X_test)
probability_predictions2 = best_tree.predict_proba(X)

matrix2 = metrics.confusion_matrix(y_test, predictions2)

print(metrics.classification_report(y_test, predictions2))

# metrics.roc_auc_score(y_test, predictions)

print(probability_predictions2[:,1])

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      1699
           1       0.07      0.06      0.07       157

    accuracy                           0.85      1856
   macro avg       0.49      0.49      0.49      1856
weighted avg       0.84      0.85      0.85      1856

[0.00415339 0.95053231 0.66510519 ... 0.00415339 0.00415339 0.00415339]


In [243]:
best_tree.score(X,y)

1.0

In [902]:
y_predicted_prob = best_tree.predict_proba(X)
pred = pd.DataFrame({'is_gone': y_predicted_prob[:,1]})
pred.head(10)


,is_gone
0,0.000000
1,1.000000
2,0.100000
3,0.921429
4,0.033333
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.016667


In [903]:
result = final_table[['user_id']]
df_c = pd.concat([result, pred], axis=1)
df_c.head(20)

,user_id,is_gone
0,12,0.000000
1,13,1.000000
2,15,0.100000
3,21,0.921429
4,35,0.033333
5,45,0.000000
6,55,0.000000
7,58,0.000000
8,62,0.000000
9,74,0.016667


In [904]:
df_c.query('user_id == 1619')

,user_id,is_gone
171,1619,0.0


ЗАПИСЫВАЕМ СОЗДАННЫЙ ДАТАФРЕЙМ В ФАЙЛ

In [905]:
df_c.to_csv(r'/Users/avyny/Desktop/result_new5.csv', index = None, header=True)